# ETL

## Settings

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import helpers.settings as sts
sts.print_settings(sts)

DATASET_TRAIN_FILENAME : dataset_train.parquet
DATASET_VALIDATION_FILENAME : dataset_validation.parquet
ETL_VERSION : 0.0.1
MODEL_FILENAME : model_0.0.1.pkl
MODEL_VERSION : 0.0.1
PREPROCESSOR_FILENAME : preprocessor_0.0.1.pkl
color : <class 'helpers.settings.color'>
print_settings : <function print_settings at 0x7facc62a7b80>



## Imports

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (
    classification_report,
    plot_precision_recall_curve,
    precision_recall_curve,
    average_precision_score,
)
from xgboost import XGBClassifier
import numpy as np
import pickle

##  Load dataset

In [5]:
df_train = pd.read_parquet(f"data/{sts.DATASET_TRAIN_FILENAME}")

In [6]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

## Load preprocessor

In [7]:
with open(f"data/{sts.PREPROCESSOR_FILENAME}","rb") as file:
    preprocessor = pickle.load(file)

In [9]:
pd.DataFrame(preprocessor.fit_transform(X_train,y_train), columns=preprocessor.final_features)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
0,0.043373,-0.017715,0.248458,0.363272,0.026367,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,0.000000,0.0,-0.217391,-0.871195,133.819774,0.0
1,0.043373,-0.017715,-0.841560,-0.073486,-0.215596,-0.439822,-0.041703,-0.002433,-0.528348,0.076444,-0.057213,0.606423,0.0,0.000000,0.773262,0.529734,0.0
2,0.043373,0.028455,0.248458,0.363272,-0.215596,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,0.000000,0.0,5.000000,-0.882108,133.819774,0.0
3,0.043373,0.028455,0.248458,-0.027511,0.026367,-0.013086,-0.041703,-0.002433,-0.528348,-0.033699,-0.057213,-0.193711,2.0,-0.434783,0.801838,0.333090,2.0
4,0.043373,-0.017715,0.248458,0.363272,-0.341495,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,0.000000,0.0,-0.434783,-1.174038,133.819774,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30983,0.043373,-0.017715,0.248458,0.363272,0.026367,0.174109,-0.041703,-0.002433,0.112936,0.076444,0.447095,0.000000,0.0,-0.434783,-1.123205,133.819774,-1.0
30984,0.043373,-0.017715,0.248458,-0.073486,0.026367,-0.013086,-0.041703,-0.002433,0.112936,-0.033699,-0.057213,-0.431977,1.0,-0.217391,0.006605,-2.443999,1.0
30985,0.043373,-0.017715,0.248458,-0.027511,0.026367,-0.013086,-0.041703,-0.002433,-0.528348,0.076444,-0.057213,-0.305770,0.0,-0.782609,-0.316485,-0.632616,0.0
30986,0.043373,-0.017715,0.248458,0.363272,0.026367,-0.439822,-0.041703,-0.002433,0.112936,0.076444,-0.057213,0.000000,0.0,0.000000,-0.869759,133.819774,0.0
